# Chest-prediction

## data cleaning

### PRELIMINARY ACTION !

⚠️ Please go to ➤ https://drive.google.com/file/d/1lLrHbpUQE-Kd-jZ68Uk7SFwawbzqf6Av/view?usp=drive_link

and download the dataset.

Put the zip file into your "*raw_data*" folder

### Code

In [1]:
import numpy as np
import pandas as pd
import pdb

import os
from pathlib import Path
from PIL import Image

#### Loading images

In [2]:
PROJECT_NAME = "chest-predictor" #to be adapted depending on the name of the Project Name in your system
NUMBER_OF_IMAGES = 100 # nbr of images to be loaded or 'full' to load the entire dataset (+100k images)

In [3]:
USERNAME = os.environ.get('USER')

In [4]:
LOCAL_DATA_PATH = Path(f"/Users/{USERNAME}/code/sachamagier/{PROJECT_NAME}/raw_data/resized_dataset")

In [5]:
print(f"LOCAL_DATA_PATH: {LOCAL_DATA_PATH}")

LOCAL_DATA_PATH: /Users/arnodebelle/code/sachamagier/chest-predictor/raw_data/resized_dataset


In [6]:
def loading_data():
    """This function either get all the images if the user set NUMBER_OF_IMAGES
    to 'full' or the number of imgaes otherwise """

    images_data = []

    # Define the path to the folder
    folder_path = f'../raw_data/resized_dataset/images/set_full/'

    
    # Get a list of all files in the folder
    file_list = os.listdir(folder_path)

    # Filter the list to only include image files
    image_files = [f for f in file_list if f.endswith('.png') or f.endswith('.jpg') or f.endswith('.jpeg')]


    # Loop through the first NUMBER_OF_IMAGES
    for i, image_file in enumerate(image_files):

        # Stop the loop after NUMBER_OF_IMAGES iterations
        if i == NUMBER_OF_IMAGES:
            break

        # Open the image file
        with Image.open(folder_path + image_file) as image:
            # Add the image to the list
            images_data.append((image_file, np.array(image)))

    return images_data

In [7]:
import urllib.request
import zipfile
import shutil


if LOCAL_DATA_PATH.is_dir():
    print("Load local data...")
    # loading data into data
    images_data = loading_data()
else:
    print("Unziping file and loading the data...")

    output_path = "../raw_data/resized_dataset.zip"
    # unzip the file
    with zipfile.ZipFile(output_path, "r") as zip_ref:
        for file_info in zip_ref.infolist():
            zip_ref.extract(file_info, "../raw_data/")
            
    if Path("../raw_data/__MACOSX").is_dir():
        # remove the __MACOSX folder if it exists
        shutil.rmtree("../raw_data/__MACOSX")
        
    # remove the zip file
    os.remove(output_path)
    images_data = loading_data()

Load local data...


In [10]:
# Create a dataframe from the list of images and their indices
images_df = pd.DataFrame(images_data, columns=['Image Index', 'image'])

# Set the index of the dataframe to the 'Image Index' column
images_df = images_df.set_index('Image Index').sort_index(ascending=True)

In [13]:
images_df['image'].shape

(100,)

In [11]:

# Reshape the image data to have a shape of (256, 256, 1)
images_df['image'] = images_df['image'].apply(lambda x: np.reshape(x, (256, 256, 1)))

ValueError: cannot reshape array of size 262144 into shape (256,256,1)

#### Loading labels data

In [ ]:
labels_df = pd.read_csv('../raw_data/resized_dataset/Data_Entry_2017.csv')

In [ ]:
labels_df.head()

#### Merging Images with labels and creating a new DF

In [ ]:
# Merge the image_df and labels_df dataframes on the 'Image Index' column
merged_df = pd.merge(images_df, labels_df[['Image Index', 'Finding Labels']], left_index=True, right_on='Image Index', how='inner')

# Rename the 'Finding Labels' column to 'labels'
merged_df = merged_df.rename(columns={'Finding Labels': 'labels'})

# Set the index of the dataframe to the 'Image Index' column
merged_df = merged_df.set_index('Image Index').sort_index(ascending=True)

In [ ]:
merged_df.head()

#### Checking data

In [ ]:
len(images_data)

In [ ]:
images_df.shape

In [ ]:
images_df['image'][0].shape

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,15))
for i in range(8):
    plt.subplot(1,8,i+1)
    plt.imshow(images_df['image'][i], cmap='gray')

In [ ]:
images_df['image'][0]